In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

### Stack / unstack

In [2]:
np.random.seed(17)
df = pd.DataFrame({'col_1':['item_1', 'item_2', 'item_3', 'item_4', 'item_5'], 
                   'col_2': np.random.randint(0, 10, 5), 
                   'col_3': np.random.randint(0, 10, 5)})
df

,col_1,col_2,col_3
0,item_1,1,6
1,item_2,6,4
2,item_3,6,7
3,item_4,9,4
4,item_5,0,7


In [3]:
df_stacked = df.stack()
df_stacked

0  col_1    item_1
   col_2         1
   col_3         6
1  col_1    item_2
   col_2         6
   col_3         4
2  col_1    item_3
   col_2         6
   col_3         7
3  col_1    item_4
   col_2         9
   col_3         4
4  col_1    item_5
   col_2         0
   col_3         7
dtype: object

Индексам также можно присвоить названия:

In [4]:
df_stacked.index.names = ['id', 'column']
df_stacked

id  column
0   col_1     item_1
    col_2          1
    col_3          6
1   col_1     item_2
    col_2          6
    col_3          4
2   col_1     item_3
    col_2          6
    col_3          7
3   col_1     item_4
    col_2          9
    col_3          4
4   col_1     item_5
    col_2          0
    col_3          7
dtype: object

В качестве аргументов stack можно передать два параметра.

- level – отвечает за уровень, по которому будет проведена стыковка
- dropna – нужно ли убрать ряды с пропущенными значениями 

In [9]:
#распаковка
df_stacked.unstack()

column,col_1,col_2,col_3
id,,,
0,item_1,1,6
1,item_2,6,4
2,item_3,6,7
3,item_4,9,4
4,item_5,0,7


In [13]:
df_stacked.unstack(0)

id,0,1,2,3,4
column,,,,,
col_1,item_1,item_2,item_3,item_4,item_5
col_2,1,6,6,9,0
col_3,6,4,7,4,7


In [14]:
df_stacked.unstack(1)

column,col_1,col_2,col_3
id,,,
0,item_1,1,6
1,item_2,6,4
2,item_3,6,7
3,item_4,9,4
4,item_5,0,7


### XS
Значения конкретного уровня индексов можно получить используя метод .xs(), передав ему интересующее нас значение индекса и уровень

In [8]:
df_stacked.xs('col_1', level = 'column')

id
0    item_1
1    item_2
2    item_3
3    item_4
4    item_5
dtype: object

### MELT()
С помощью метода melt можно "расплавить" данные и привести их к длинному формату. Так, одна или несколько колонок помещаются в качестве идентификационных переменных, а остальные столбцы считаются измеряемыми переменными

In [76]:
df3 = pd.DataFrame({'name': ['Persik', 'Brownie'], 'type': ['cat', 'dog'],
                    'color': ['ginger', 'white'], 'height': [17, 30], 
                    'weight': [3.4, 4.3]})
df3

,name,type,color,height,weight
0,Persik,cat,ginger,17,3.4
1,Brownie,dog,white,30,4.3


In [77]:
df3.melt().head()

,variable,value
0,name,Persik
1,name,Brownie
2,type,cat
3,type,dog
4,color,ginger


Для изменения названий полученных столбцов используются параметры var_name и value_name. Например, передаем в качестве id_vars имена, для значений (value_vars) используем только три колонки и изменяем названия новых колонок:

In [78]:
df3.melt(id_vars=['name'], value_vars=['type', 'color', 'height'], 
         var_name='characteristics', value_name='value')

,name,characteristics,value
0,Persik,type,cat
1,Brownie,type,dog
2,Persik,color,ginger
3,Brownie,color,white
4,Persik,height,17
5,Brownie,height,30


### pd.wide_to_long

Еще один вариант для перевода данных из широкого формата в длинный — pd.wide_to_long(). 

Предположим, мы собрали побольше данных о котике Персике и пёсике Брауни, и добавили данные о весе и росте уже за два года:

      name type  AvgHeight_2019  AvgHeight_2020  AvgWeight_2019  AvgWeight_2020   color 
0   Persik  cat       17.077963       17.134233             3.4          3.5545  ginger
1  Brownie  dog       30.673324       30.674466             4.3          4.5716   white    

Посмотрим на аргументы функции более подробно.

- data — датафрейм
- stubnames — части названий переменных, которые мы хотим преобразовать из широкого формата в длинный
- i — переменные, которые не трансформируются, и в результате помещаются в индексы
- j — имя новой переменной
- sep — разделитель (между параметром и значением)
В данном случае у нас есть две общих характеристики, отвечающих за рост и вес в конкретный год. Названия соответствующих переменных состоят из AvgHeight / AvgWeight и года, поэтому в stubnames мы передаем список параметров (вес и рост), а оставшаяся часть названия (2018, 2019) будет использована в качестве значений новой переменной year. Столбцы type и name помещаем в индексы, а параметр color оставляем обычной колонкой.

### explode()

In [101]:
df = pd.DataFrame({'A': [[1, 2, 3], 
                         'kitten', 
                         [], 
                         ['kitten', 'puppy']], 
                   'B': 1})
df

,A,B
0,"[1, 2, 3]",1
1,kitten,1
2,[],1
3,"[kitten, puppy]",1


In [102]:
df.explode('A')

,A,B
0,1,1
0,2,1
0,3,1
1,kitten,1
2,NaN,1
3,kitten,1
3,puppy,1


### resample

Теперь посмотрим, какие возможности pandas предоставляет для работы с временными рядами! Один из наиболее часто используемых и удобных методов — .resample(), позволяющий преобразовать данные и применить к ним другой метод (sum(), size() и пр.). Таким образом, можно рассчитать показатели, например, за весь день, неделю, месяц и т.п. С полным списком возможных значений можно ознакомиться здесь https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects.

Например, посчитать сумму показателя по дням, имея данные по часам, можно следующим образом:

data.resample(rule='D').sum()

In [ ]:
data = pd.read_csv('data_lessons/bikes_q1_sample.csv', sep = ',')

In [122]:
data['start_time'] = pd.to_datetime(data['start_time'])
data = data.set_index('start_time')
data.resample(rule='D').sum()

,trip_id,bikeid,from_station_id,to_station_id,birthyear
start_time,,,,,
2018-01-01,3349561242,679058,42159,38962,364973.0
2018-01-02,13311733637,2872211,134037,131343,1500158.0
2018-01-03,19242971711,4080211,194219,195848,2145691.0
2018-01-04,19509618787,4189136,201018,196943,2175025.0
2018-01-05,17617932909,3561635,178645,176522,1967491.0
...,...,...,...,...,...
2018-03-27,49519967691,9874012,500510,506768,5044453.0
2018-03-28,65433876386,12722993,653508,654158,6473941.0
2018-03-29,50356503173,10194603,505335,503307,5160515.0


In [131]:
#посчитаем кол-во в разрезе индекса - дня
data.resample(rule='D').size()

start_time
2018-01-01     191
2018-01-02     759
2018-01-03    1097
2018-01-04    1112
2018-01-05    1004
              ... 
2018-03-27    2756
2018-03-28    3640
2018-03-29    2800
2018-03-30    2913
2018-03-31    1844
Freq: D, Length: 90, dtype: int64

### Стиль

Помимо методов для работы с данными, pandas включает в себя возможности для форматирования таблиц!

Например: 

- df.style.highlight_null() – подсветить ячейки с пропущенными значениями 
- df.style.highlight_max() – подсветить ячейки с максимальными значениями по колонкам
- df.style.highlight_min() – подсветить ячейки с минимальными значениями по колонкам
- df.style.applymap(func) – применить стилевую функцию к каждой ячейке датафрэйма
- df.style.apply(func, axis, subset) – применить стилевую функцию к каждой колонке/строке в зависимости от axis, subsetпозволяет выбрать часть колонок для оформления
- render() – после декорирования возвращает HTML, описывающий табличку

In [166]:
np.random.seed(77)
df = pd.DataFrame({'A': list(range(5)), 
                   'B': np.random.randint(0, 10, 5),
                   'C': np.random.randint(-10, 10, 5), 
                   'D': np.random.randint(-10, 100, 5)})
df

,A,B,C,D
0,0,7,-10,49
1,1,4,-3,90
2,2,4,2,44
3,3,5,9,26
4,4,8,-10,37


In [167]:
#спрятать индекс
df.style.hide_index()

A,B,C,D
0,7,-10,49
1,4,-3,90
2,4,2,44
3,5,9,26
4,8,-10,37


In [168]:
#добавляем название таблицы
df.style.hide_index().set_caption('Cool table')

A,B,C,D
0,7,-10,49
1,4,-3,90
2,4,2,44
3,5,9,26
4,8,-10,37


In [169]:
# highlight_max – подсвечивает (выделяет) цветом наибольшее значение. Можно применить либо к каждой строке (axis=0/'index'), либо к каждой колонке (axis=1/'columns').

df.style.highlight_max(axis=1)

,A,B,C,D
0,0,7,-10,49
1,1,4,-3,90
2,2,4,2,44
3,3,5,9,26
4,4,8,-10,37


In [170]:
df.style.highlight_max(axis='index')


,A,B,C,D
0,0,7,-10,49
1,1,4,-3,90
2,2,4,2,44
3,3,5,9,26
4,4,8,-10,37


In [171]:
# Аналогичная функция для подсветки минимальных значений – highlight_min().

df.style.highlight_min()

,A,B,C,D
0,0,7,-10,49
1,1,4,-3,90
2,2,4,2,44
3,3,5,9,26
4,4,8,-10,37


In [172]:
# background_gradient – раскрашивает ячейки в зависимости от их значений. В итоге получается что-то похожее на heatmap (тепловую карту). Например:

(df.style
 .highlight_min('A', color='red')
 .highlight_max('B', color='orange')
 .background_gradient(subset=['C','D'],cmap='viridis')
)

# Здесь мы сначала выделяем красным минимальное значение в столбце A (highlight_min), затем – оранжевым максимальное в колонке B (highlight_max), и применяем background_gradient для C и D, указав палитру viridis. 

,A,B,C,D
0,0,7,-10,49
1,1,4,-3,90
2,2,4,2,44
3,3,5,9,26
4,4,8,-10,37


### style.bar

Визуализировать значения можно прямо в таблице с помощью .bar(). Данный метод принимает несколько аргументов:

subset – для каких колонок нужно построить небольшой барплот \
color – цвет 

In [173]:
df.style.bar(subset=['C', 'D'], color='#67A5EB')

,A,B,C,D
0,0,7,-10,49
1,1,4,-3,90
2,2,4,2,44
3,3,5,9,26
4,4,8,-10,37


In [174]:
# align –  как выровнять столбики (mid – центр ячейки в (max-min)/2; zero – ноль находится в центре ячейки; left – минимальное значение находится в левой части ячейки)
df.style.bar(subset=['C', 'D'], color='#67A5EB', align='mid')

,A,B,C,D
0,0,7,-10,49
1,1,4,-3,90
2,2,4,2,44
3,3,5,9,26
4,4,8,-10,37


In [175]:
(df
 .style
 .hide_index()
 .bar(subset=['C'], align='mid',color=['#d65f5f', '#5fba7d'])
)

A,B,C,D
0,7,-10,49
1,4,-3,90
2,4,2,44
3,5,9,26
4,8,-10,37


### Форматирование чисел

In [176]:
# генерируем данные
df = pd.DataFrame({'A': np.linspace(1, 10, 5)})
df = pd.concat([df, pd.DataFrame(np.random.randn(5, 4), columns=list('BCDE'))],axis=1)
df['F'] = np.random.choice(['A', 'B'], size=5)
df.iloc[3, 3] = np.nan 
df.iloc[0, 2] = np.nan 
df


,A,B,C,D,E,F
0,1.00,0.797939,NaN,-1.652119,0.717119,B
1,3.25,0.977228,-1.040849,-0.643520,-0.112520,A
2,5.50,-0.314166,1.627440,-0.361227,-0.173046,B
3,7.75,-1.951309,-0.978210,NaN,-1.178379,A
4,10.00,-0.515551,-0.063015,-0.559371,0.796697,A


Форматируем:

- оставляем только 2 знака после точки
- добавляем знак + для положительных значений
- применяем ко всем колонкам, кроме F

In [177]:
df.style.format("{:+.2f}", subset=df.columns.drop('F'))

,A,B,C,D,E,F
0,+1.00,+0.80,+nan,-1.65,+0.72,B
1,+3.25,+0.98,-1.04,-0.64,-0.11,A
2,+5.50,-0.31,+1.63,-0.36,-0.17,B
3,+7.75,-1.95,-0.98,+nan,-1.18,A
4,+10.00,-0.52,-0.06,-0.56,+0.80,A


In [178]:
# Также можем скрыть индексы и добавить название:

(df.style
 .format({'B': "{:0<4.0f}", 'D': '{:+.2f}'})
 .hide_index()
 .set_caption('Новая таблица'))

A,B,C,D,E,F
1.000000,1000,nan,-1.65,0.717119,B
3.250000,1000,-1.040849,-0.64,-0.112520,A
5.500000,-000,1.627440,-0.36,-0.173046,B
7.750000,-200,-0.978210,+nan,-1.178379,A
10.000000,-100,-0.063015,-0.56,0.796697,A


In [179]:
# И при желании импортировать в Excel (но не всё форматирование переносится):


(df.style
 .bar(align='mid', color=['#d65f5f', '#5fba7d'])
 .to_excel('styled.xlsx', engine='openpyxl')
)

In [194]:
data = pd.read_csv('data_lessons/user_retention.csv', sep = ',')

In [195]:
data = data.set_index('Cohort')

In [183]:
data.head()

,Cohort,0,1,2,3,4,5,6,7,8,9,10,11,12
0,2010-12,1.0,0.381857,0.334388,0.387131,0.359705,0.396624,0.379747,0.354430,0.354430,0.394515,0.373418,0.500000,0.274262
1,2011-01,1.0,0.239905,0.282660,0.242280,0.327791,0.299287,0.261283,0.256532,0.311164,0.346793,0.368171,0.149644,NaN
2,2011-02,1.0,0.247368,0.192105,0.278947,0.268421,0.247368,0.255263,0.281579,0.257895,0.313158,0.092105,NaN,NaN
3,2011-03,1.0,0.190909,0.254545,0.218182,0.231818,0.177273,0.263636,0.238636,0.288636,0.088636,NaN,NaN,NaN
4,2011-04,1.0,0.227425,0.220736,0.210702,0.207358,0.237458,0.230769,0.260870,0.083612,NaN,NaN,NaN,NaN


In [196]:
ur_style = (data
            .style
            .set_caption('User retention by cohort')  # добавляем подпись
            .background_gradient(cmap='viridis')  # раскрашиваем ячейки по столбцам
            .highlight_null('white')  # делаем белый фон для значений NaN
            .format("{:.2%}", na_rep=""))  # числа форматируем как проценты, NaN заменяем на пустоту
ur_style

/Users/admin/opt/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


,0,1,2,3,4,5,6,7,8,9,10,11,12
Cohort,,,,,,,,,,,,,
2010-12,100.00%,38.19%,33.44%,38.71%,35.97%,39.66%,37.97%,35.44%,35.44%,39.45%,37.34%,50.00%,27.43%
2011-01,100.00%,23.99%,28.27%,24.23%,32.78%,29.93%,26.13%,25.65%,31.12%,34.68%,36.82%,14.96%,
2011-02,100.00%,24.74%,19.21%,27.89%,26.84%,24.74%,25.53%,28.16%,25.79%,31.32%,9.21%,,
2011-03,100.00%,19.09%,25.45%,21.82%,23.18%,17.73%,26.36%,23.86%,28.86%,8.86%,,,
2011-04,100.00%,22.74%,22.07%,21.07%,20.74%,23.75%,23.08%,26.09%,8.36%,,,,
2011-05,100.00%,23.66%,17.20%,17.20%,21.51%,24.37%,26.52%,10.39%,,,,,
2011-06,100.00%,20.85%,18.72%,27.23%,24.68%,33.62%,10.21%,,,,,,
2011-07,100.00%,20.94%,20.42%,23.04%,27.23%,11.52%,,,,,,,
2011-08,100.00%,25.15%,25.15%,25.15%,13.77%,,,,,,,,


# ДЗ

In [16]:
data = pd.read_csv('data_lessons/Pokemon.csv', sep = ',')

In [17]:
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [35]:
x = pd.Series(list(data.columns)).apply(lambda x: x.replace(". ",'_'))
x = x.apply(lambda x: x.replace(" ",'_'))
x = x.apply(lambda x: x.lower())
data.columns = list(x.apply(lambda x: x.replace("#",'id')))

In [36]:
data.head()

,id,name,type_1,type_2,total,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [50]:
q = data.groupby(['generation']).legendary.value_counts().to_frame()
q

legendary
generation legendary           
1          False            160
           True               6
2          False            101
           True               5
3          False            142
           True              18
4          False            108
           True              13
5          False            150
           True              15
6          False             74
           True               8

In [51]:
q.columns = ['legendary_counts']
q.unstack()

legendary_counts      
legendary             False True 
generation                       
1                       160     6
2                       101     5
3                       142    18
4                       108    13
5                       150    15
6                        74     8

In [70]:
g = data.groupby(['generation','type_1']).agg({'legendary':'sum'})
g

legendary
generation type_1             
1          Bug             0.0
           Dragon          0.0
           Electric        1.0
           Fairy           0.0
           Fighting        0.0
...                        ...
6          Poison          0.0
           Psychic         2.0
           Rock            2.0
           Steel           0.0
           Water           0.0

[98 rows x 1 columns]

In [75]:
g.idxmax()

legendary    (3, Dragon)
dtype: object

In [61]:
d = data.groupby(['generation','type_1']).agg({'legendary':'sum'}).unstack().fillna(0)
d

legendary                                                        \
type_1           Bug Dark Dragon Electric Fairy Fighting Fire Flying Ghost   
generation                                                                   
1                0.0  0.0    0.0      1.0   0.0      0.0  1.0    0.0   0.0   
2                0.0  0.0    0.0      1.0   0.0      0.0  2.0    0.0   0.0   
3                0.0  0.0    6.0      0.0   0.0      0.0  0.0    0.0   0.0   
4                0.0  1.0    0.0      0.0   0.0      0.0  1.0    0.0   2.0   
5                0.0  0.0    5.0      2.0   0.0      0.0  0.0    2.0   0.0   
6                0.0  1.0    1.0      0.0   1.0      0.0  1.0    0.0   0.0   

                                                                     
type_1     Grass Ground  Ice Normal Poison Psychic Rock Steel Water  
generation                                                           
1            0.0    0.0  1.0    0.0    0.0     3.0  0.0   0.0   0.0  
2            0.0    0.0  0.0    0.0    0.0     1.0  0.0   0.0   1.0  
3            0.0    2.0  1.0    0.0    0.0     4.0  1.0   2.0   2.0  
4            2.0    0.0  0.0    2.0    0.0     3.0  0.0   1.0   1.0  
5            1.0    2.0  0.0    0.0    0.0     1.0  1.0   1.0   0.0  
6            0.0    0.0  0.0    0.0    0.0     2.0  2.0   0.0   0.0

In [79]:
data = pd.read_csv('data_lessons/superheroes_power_matrix.csv')

In [80]:
data.head()

,Name,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,...,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient
0,3-D Man,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [96]:
#
superheroes_long = data.melt(id_vars=['Name'],var_name='superpower')
superheroes_long

,Name,superpower,value
0,3-D Man,Agility,True
1,A-Bomb,Agility,False
2,Abe Sapien,Agility,True
3,Abin Sur,Agility,False
4,Abomination,Agility,False
...,...,...,...
111384,Yellowjacket II,Omniscient,False
111385,Ymir,Omniscient,False
111386,Yoda,Omniscient,False
111387,Zatanna,Omniscient,False


In [97]:
#поместим superpower в листы
superheroes_long.query("value == True").groupby(['Name'])['superpower'].apply(list)

Name
3-D Man              [Agility, Super Strength, Stamina, Super Speed]
A-Bomb             [Accelerated Healing, Durability, Longevity, S...
Abe Sapien         [Agility, Accelerated Healing, Cold Resistance...
Abin Sur                                        [Lantern Power Ring]
Abomination        [Accelerated Healing, Intelligence, Super Stre...
                                         ...                        
Yellowjacket II               [Flight, Energy Blasts, Size Changing]
Ymir               [Cold Resistance, Durability, Longevity, Super...
Yoda               [Agility, Stealth, Danger Sense, Marksmanship,...
Zatanna            [Cryokinesis, Telepathy, Magic, Fire Control, ...
Zoom               [Super Speed, Intangibility, Time Travel, Time...
Name: superpower, Length: 667, dtype: object

In [99]:
superheroes_long.query("value == True").groupby(['Name'])['superpower'].apply(list).to_frame().reset_index()

,Name,superpower
0,3-D Man,"[Agility, Super Strength, Stamina, Super Speed]"
1,A-Bomb,"[Accelerated Healing, Durability, Longevity, S..."
2,Abe Sapien,"[Agility, Accelerated Healing, Cold Resistance..."
3,Abin Sur,[Lantern Power Ring]
4,Abomination,"[Accelerated Healing, Intelligence, Super Stre..."
...,...,...
662,Yellowjacket II,"[Flight, Energy Blasts, Size Changing]"
663,Ymir,"[Cold Resistance, Durability, Longevity, Super..."
664,Yoda,"[Agility, Stealth, Danger Sense, Marksmanship,..."
665,Zatanna,"[Cryokinesis, Telepathy, Magic, Fire Control, ..."


In [104]:
data = pd.read_csv('data_lessons/bikes_q1_sample.csv', sep = ',')

In [105]:
data.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,17617135,2018-01-22 20:04:31,2018-01-22 20:11:53,1131,442.0,471,Francisco Ave & Foster Ave,468,Budlong Woods Library,Subscriber,Female,1949.0
1,17897619,2018-03-16 19:47:59,2018-03-16 20:04:00,6146,961.0,296,Broadway & Belmont Ave,253,Winthrop Ave & Lawrence Ave,Subscriber,Male,1988.0
2,17881307,2018-03-14 18:49:20,2018-03-14 18:54:38,3847,318.0,260,Kedzie Ave & Milwaukee Ave,503,Drake Ave & Fullerton Ave,Subscriber,Male,1987.0
3,17881130,2018-03-14 18:33:48,2018-03-14 19:07:40,1483,"2,032.0",199,Wabash Ave & Grand Ave,199,Wabash Ave & Grand Ave,Subscriber,Male,1990.0
4,17686289,2018-02-05 17:39:14,2018-02-05 17:46:13,6391,419.0,596,Benson Ave & Church St,605,University Library (NU),Subscriber,Male,1992.0


In [114]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174215 entries, 0 to 174214
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   trip_id            174215 non-null  int64         
 1   start_time         174215 non-null  datetime64[ns]
 2   end_time           174215 non-null  object        
 3   bikeid             174215 non-null  int64         
 4   tripduration       174215 non-null  object        
 5   from_station_id    174215 non-null  int64         
 6   from_station_name  174215 non-null  object        
 7   to_station_id      174215 non-null  int64         
 8   to_station_name    174215 non-null  object        
 9   usertype           174215 non-null  object        
 10  gender             164161 non-null  object        
 11  birthyear          164236 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(6)
memory usage: 15.9+ MB


In [119]:
data['start_time'] = pd.to_datetime(data['start_time'])
data = data.set_index('start_time')
data.resample(rule='D').sum()

,trip_id,bikeid,from_station_id,to_station_id,birthyear
start_time,,,,,
2018-01-01,3349561242,679058,42159,38962,364973.0
2018-01-02,13311733637,2872211,134037,131343,1500158.0
2018-01-03,19242971711,4080211,194219,195848,2145691.0
2018-01-04,19509618787,4189136,201018,196943,2175025.0
2018-01-05,17617932909,3561635,178645,176522,1967491.0
...,...,...,...,...,...
2018-03-27,49519967691,9874012,500510,506768,5044453.0
2018-03-28,65433876386,12722993,653508,654158,6473941.0
2018-03-29,50356503173,10194603,505335,503307,5160515.0


In [132]:
data.resample(rule='D').size().max()

4196

In [149]:
data = pd.read_csv('data_lessons/bikes_april.csv', sep = ',')

In [136]:
data.head()

,start_time,trip_id,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,2018-04-01 00:10:23,18000531,2018-04-01 00:22:12,5065,709.0,228,Damen Ave & Melrose Ave,219,Damen Ave & Cortland St,Subscriber,Male,1983.0
1,2018-04-01 00:15:49,18000533,2018-04-01 00:19:47,4570,238.0,128,Damen Ave & Chicago Ave,130,Damen Ave & Division St,Subscriber,Male,1978.0
2,2018-04-01 00:17:00,18000534,2018-04-01 00:22:53,1323,353.0,130,Damen Ave & Division St,69,Damen Ave & Pierce Ave,Subscriber,Male,1991.0
3,2018-04-01 00:20:00,18000536,2018-04-01 00:26:22,2602,382.0,121,Blackstone Ave & Hyde Park Blvd,351,Cottage Grove Ave & 51st St,Subscriber,Female,1992.0
4,2018-04-01 00:23:19,18000538,2018-04-01 00:35:01,4213,702.0,31,Franklin St & Chicago Ave,180,Ritchie Ct & Banks St,Subscriber,Male,1985.0


In [160]:
data['start_time'] = pd.to_datetime(data['start_time'])
i = data.groupby(['usertype','start_time'], as_index = False).agg({'trip_id':'count'})
i['start_time'] = pd.to_datetime(i['start_time'])
i = i.set_index(['start_time'])
i = i.query("usertype == 'Subscriber'")
i = i.resample(rule='D').sum()

In [161]:
i.head()

,trip_id
start_time,
2018-04-01,825
2018-04-02,2841
2018-04-03,1873
2018-04-04,2253
2018-04-05,2502


In [165]:
i.loc['2018-04-01':'2018-04-05']

,trip_id
start_time,
2018-04-01,825
2018-04-02,2841
2018-04-03,1873
2018-04-04,2253
2018-04-05,2502
